In [ ]:
# ==============================================================================
# --- 1. 라이브러리 임포트 ---
# ==============================================================================
import pandas as pd
import os
import json
from pathlib import Path
import time
from datetime import datetime, timedelta
import re
import csv
import requests
from bs4 import BeautifulSoup
import shutil

# Selenium 관련 라이브러리
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [ ]:
# ==============================================================================
# --- 2. 설정 (이 부분만 사용자에 맞게 수정하세요) ---
# ==============================================================================
# 1. 파일 경로 및 이름 설정
#    - 현재 .ipynb 파일이 AIDA/ 폴더에 있다고 가정합니다.
#    - 경로가 다르면 이 부분을 수정해주세요.
DATA_DIR = "data"
PROCESSED_DIR = "processed"
INPUT_FILENAME = "news_cluster_w_event_name.csv"
FINAL_OUTPUT_FILENAME = "Comment_all.csv"

TEMP_OUTPUT_DIR = "crawled_temp_output"
PROGRESS_FILENAME = "crawling_progress.json"

# 2. 크롤링 기간 설정
START_DATE = "2025.08.14"
END_DATE = (datetime.now() - timedelta(days=1)).strftime("%Y.%m.%d") # 어제까지

# 3. 크롤링 세부 설정
MAX_PAGES_PER_QUERY = 10
INTERMEDIATE_SAVE_COUNT = 200

# 4. 수동 재시작 (필요할 때만 사용, 평소에는 None)
MANUAL_RESUME_CATEGORY = None
MANUAL_RESUME_INDEX = None
# ===================================================================

In [ ]:
# ==============================================================================
# --- 3. 기능 함수 정의 ---
# ==============================================================================

def setup_driver():
    """크롤링에 사용할 웹 드라이버를 설정합니다."""
    try:
        print("🚀 Selenium WebDriver를 설정합니다...")
        options = Options()
        # options.add_argument("--headless") # 안정성을 위해 실제 브라우저 창을 띄웁니다.
        options.add_argument("--start-maximized")
        options.add_argument("--disable-gpu")
        options.add_argument("--disable-infobars")
        options.add_argument("--disable-extensions")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        print("✅ WebDriver 설정 완료. (브라우저 창이 나타납니다)")
        return driver
    except Exception as e:
        print(f"❌ WebDriver 설정 중 오류 발생: {e}"); return None

def save_progress(progress_file: Path, category: str, query_index: int):
    """현재 진행 상황을 JSON 파일에 저장합니다."""
    progress = {'last_completed_category': category, 'last_completed_query_index': query_index}
    with open(progress_file, 'w', encoding='utf-8') as f:
        json.dump(progress, f, ensure_ascii=False, indent=4)

def load_progress(progress_file: Path):
    """저장된 진행 상황을 JSON 파일에서 불러옵니다."""
    try:
        with open(progress_file, 'r', encoding='utf-8') as f:
            progress = json.load(f)
            print(f"✅ 이전 진행 상황 로드: {progress}")
            return progress.get('last_completed_category'), progress.get('last_completed_query_index', -1)
    except (FileNotFoundError, json.JSONDecodeError, KeyError):
        return None, -1

def format_naver_date(date_str: str) -> str:
    """네이버 뉴스 댓글의 날짜 형식을 'YYYYMMDDHHMMSS' 문자열로 변환합니다."""
    now = datetime.now()
    date_str = str(date_str).strip()
    try:
        if "방금 전" in date_str or "초 전" in date_str: return str(now.strftime('%Y%m%d%H%M%S'))
        if "분 전" in date_str:
            min_ago = int(re.search(r'\d+', date_str).group())
            return str((now - timedelta(minutes=min_ago)).strftime('%Y%m%d%H%M%S'))
        if "시간 전" in date_str:
            hour_ago = int(re.search(r'\d+', date_str).group())
            return str((now - timedelta(hours=hour_ago)).strftime('%Y%m%d%H%M%S'))
        if "어제" in date_str:
            target_time = now - timedelta(days=1)
            if time_match := re.search(r'(\d{2}):(\d{2})', date_str):
                hour, minute = map(int, time_match.groups())
                target_time = target_time.replace(hour=hour, minute=minute, second=0)
            return str(target_time.strftime('%Y%m%d%H%M%S'))
        if re.match(r'\d{4}\.\d{2}\.\d{2}\. \d{2}:\d{2}', date_str):
            return str(datetime.strptime(date_str, '%Y.%m.%d. %H:%M').strftime('%Y%m%d%H%M00'))
        if re.match(r'\d{4}\.\d{2}\.\d{2}\.', date_str):
            return str(datetime.strptime(date_str, '%Y.%m.%d.').strftime('%Y%m%d000000'))
    except (ValueError, AttributeError): pass
    return str(now.strftime('%Y%m%d%H%M%S'))

def crawl_article_content(article_url: str, headers: dict) -> str:
    """기사 본문을 크롤링합니다."""
    try:
        response = requests.get(article_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        content_area = soup.select_one('#dic_area, #articeBody, #newsct_article')
        if content_area:
            for tag in content_area.select('script, style, .reporter_area'): tag.decompose()
            return content_area.get_text(strip=True, separator='\n')
        return ""
    except Exception: return ""

def crawl_comments(driver: webdriver.Chrome, article_url: str) -> list:
    """댓글 전용 URL로 접속하여 댓글을 크롤링합니다."""
    if not driver: return []
    match = re.search(r'/article/(\d+)/(\d+)', article_url)
    if not match: return []
    
    press_id, article_id = match.groups()
    comment_page_url = f"https://n.news.naver.com/article/comment/{press_id}/{article_id}"
    
    try:
        driver.get(comment_page_url)
        try:
            comment_count_element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.u_cbox_count")))
            comment_count = int(re.sub(r'[^0-9]', '', comment_count_element.text))
            if comment_count == 0: return []
        except (TimeoutException, ValueError): return []
            
        click_count = 0
        while True:
            try:
                more_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.u_cbox_btn_more")))
                driver.execute_script("arguments[0].click();", more_button); click_count += 1
                print(f"    -> '더보기' ({click_count}회)", end='\r'); time.sleep(0.5)
            except (NoSuchElementException, TimeoutException):
                if click_count > 0: print("\n    -> '더보기' 완료.")
                break
                
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        contents = soup.select("span.u_cbox_contents")
        dates = soup.select("span.u_cbox_date")
        
        return [{'comment_content': c.get_text(strip=True), 'comment_date': format_naver_date(d.get_text(strip=True))} for c, d in zip(contents, dates)]
    except Exception as e:
        print(f"\n    ❌ 댓글 수집 오류: {e}"); return []

def save_to_csv(data_list: list, file_path: Path):
    """데이터를 CSV 파일에 추가 저장합니다."""
    if not data_list: return
    write_header = not file_path.exists()
    try:
        with open(file_path, 'a', newline='', encoding='utf-8-sig') as f:
            fieldnames = ['category', 'main_name', 'query', 'title', 'url', 'content', 'comment_content', 'comment_date']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            if write_header: writer.writeheader()
            writer.writerows(data_list)
    except Exception as e: print(f"\n❌ 파일 저장 오류: {e}")

def finalize_results(crawled_dir: Path, final_output_path: Path):
    """중간 저장된 모든 CSV를 합치고, 중복 제거 후 임시 폴더를 삭제합니다."""
    print("\n\n--- [최종 단계] 결과 파일 병합 및 정리 ---")
    
    csv_files = [f for f in os.listdir(crawled_dir) if f.startswith('crawled_') and f.endswith('.csv')]
    if not csv_files: print("-> 병합할 파일이 없습니다."); return

    df_list = [pd.read_csv(crawled_dir / file, dtype={'comment_date': str}) for file in csv_files]
    combined_df = pd.concat(df_list, ignore_index=True)
    
    print(f"-> 중복 제거 전: {len(combined_df)} 건")
    deduplicated_df = combined_df.drop_duplicates()
    print(f"-> 중복 제거 후: {len(deduplicated_df)} 건")
    
    final_output_path.parent.mkdir(parents=True, exist_ok=True)
    deduplicated_df.to_csv(final_output_path, index=False, encoding='utf-8-sig')
    print(f"✅ 최종 결과 저장 완료: '{final_output_path}'")
    
    try:
        shutil.rmtree(crawled_dir)
        print(f"✅ 임시 폴더 '{crawled_dir.name}' 삭제 완료.")
    except Exception as e: print(f"❗️ 임시 폴더 삭제 오류: {e}")


In [ ]:
# ==============================================================================
# --- 4. 메인 실행 로직 ---
# ==============================================================================
# (Jupyter Notebook에서는 이 부분을 하나의 셀로 분리하여 실행해도 좋습니다)

# --- 경로 준비 ---
base_path = Path(".") # 현재 .ipynb 파일이 있는 위치를 기준으로 합니다.
input_path = base_path / DATA_DIR / PROCESSED_DIR / INPUT_FILENAME
temp_output_dir = base_path / TEMP_OUTPUT_DIR
final_output_path = base_path / DATA_DIR / PROCESSED_DIR / FINAL_OUTPUT_FILENAME
progress_file = temp_output_dir / PROGRESS_FILENAME
temp_output_dir.mkdir(parents=True, exist_ok=True)

# --- 입력 파일 로드 ---
try:
    df = pd.read_csv(input_path)
    categories = df['predicted_category'].dropna().unique()
    print(f"✅ '{INPUT_FILENAME}' 파일에서 {len(categories)}개 카테고리 로드 완료.")
except (FileNotFoundError, KeyError) as e:
    print(f"❌ 입력 파일 오류: {e}")
    df = None # 오류 발생 시 df를 None으로 설정

# --- 메인 크롤링 실행 ---
if df is not None:
    if MANUAL_RESUME_CATEGORY and MANUAL_RESUME_INDEX is not None:
        save_progress(progress_file, MANUAL_RESUME_CATEGORY, MANUAL_RESUME_INDEX)
        print(f"✅ 수동 재시작 지점 설정됨: '{MANUAL_RESUME_CATEGORY}'의 {MANUAL_RESUME_INDEX + 1}번째부터")

    driver = setup_driver()
    
    if driver:
        completed_successfully = False
        try:
            start_category, start_index = load_progress(progress_file)
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;...)'} # User-Agent
            can_start_crawling = not start_category

            for category in categories:
                if not can_start_crawling and category != start_category:
                    print(f"⏩ 카테고리 [{category}] 건너뛰기")
                    continue
                can_start_crawling = True
                
                output_csv_path = temp_output_dir / f"crawled_{category}.csv"
                queries = df[df['predicted_category'] == category][['cluster_event_name', 'predicted_MAIN_NAME']].drop_duplicates().to_dict('records')
                temp_results = []
                current_start_index = start_index if category == start_category else 0
                
                for i in range(current_start_index, len(queries)):
                    item = queries[i]
                    query, main_name = item['cluster_event_name'], item['predicted_MAIN_NAME']
                    print(f"\n▶ [{category}] {i+1}/{len(queries)} - \"{query}\"")

                    for page in range(1, MAX_PAGES_PER_QUERY + 1):
                        url = f"https://search.naver.com/search.naver?where=news&query={quote_plus(query)}&start={(page-1)*10+1}&pd=3&ds={START_DATE}&de={END_DATE}"
                        try:
                            response = requests.get(url, headers=headers)
                            soup = BeautifulSoup(response.text, 'html.parser')
                            news_containers = [h.parent for h in soup.select('div[data-sds-comp="Profile"]') if h.parent]
                            if not news_containers: break

                            for container in news_containers:
                                title_tag = container.select_one('a[data-heatmap-target=".tit"]')
                                naver_news_tag = container.select_one('a[data-heatmap-target=".nav"]')
                                if not (title_tag and naver_news_tag and "n.news.naver.com" in naver_news_tag['href']): continue
                                
                                news_url = naver_news_tag['href']
                                title = title_tag.get_text(strip=True)
                                content = crawl_article_content(news_url, headers)
                                comments = crawl_comments(driver, news_url)
                                print(f"  - 기사: {title[:30]}... (댓글: {len(comments)}개)")
                                
                                for comment in comments:
                                    temp_results.append({
                                        'category': category, 'main_name': main_name, 'query': query,
                                        'title': title, 'url': news_url, 'content': content,
                                        'comment_content': comment['comment_content'], 'comment_date': comment['comment_date']
                                    })
                                
                                if len(temp_results) >= INTERMEDIATE_SAVE_COUNT:
                                    save_to_csv(temp_results, output_csv_path); temp_results.clear()
                                time.sleep(0.5)
                        except Exception as e: print(f"  ❌ 페이지 처리 오류: {e}")
                        time.sleep(1)
                    
                    save_progress(progress_file, category, i)
                
                save_to_csv(temp_results, output_csv_path)
                start_index = 0
            
            completed_successfully = True
        finally:
            if driver: driver.quit()
            if completed_successfully and progress_file.exists(): os.remove(progress_file)

        # --- 최종 결과 파일 정리 ---
        finalize_results(temp_output_dir, final_output_path)
        
        print("\n🎉 모든 작업이 완료되었습니다.")